## Inputs from the user

In [1]:
# the tickers interesting for comparison

my_symbols = ['']

## Libraries, Credentials and Requests

In [2]:
# adding libraries

import requests
import pandas as pd
import json
from requests_oauthlib import OAuth1
import datetime
import numpy as np

pd.set_option('display.max_rows', 500)

In [3]:
# credentials for access
consumer_key = ''
consumer_secret = ''
oAuth_token = ''
oAuth_token_secret = ''

# oAuth protocol for access
auth = OAuth1(consumer_key, consumer_secret, oAuth_token, oAuth_token_secret)

# uri for outputs
uri_acct = 'https://devapi.invest.ally.com/v1/accounts.json'
uri_hist = 'https://devapi.invest.ally.com/v1/accounts/'
uri_hist2 = '/history.json'
uri_stock = 'https://devapi.invest.ally.com/v1/market/ext/quotes.json'
uri_option = 'https://devapi.invest.ally.com/v1/market/options/search.json'
uri_option_strike = 'https://devapi.invest.ally.com/v1/market/options/strikes.json'
uri_option_exp = 'https://devapi.invest.ally.com/v1/market/options/expirations.json'

# API call extensions
stock_symbol = '?symbols='
symbol_call = '?symbol='
date_qry = '&query=xdate-eq%3A'

## Current Open Positions

In [4]:
accounts = requests.get(uri_acct, auth = auth).json()
my_accounts = {}
for i in accounts['response']['accounts']['accountsummary']:
    my_accounts[i['account']] = i['accountbalance']['money']['cashavailable']
    
rollover_acct, personal_acct, rollover_balance, personal_balance = 0, 0, 0, 0
personal_hold, rollover_hold = [], []
for key, balance in  my_accounts.items():
    if str(key)[-2:] == '53':
        rollover_acct = key
        rollover_balance = balance
    elif str(key)[-2:] == '81':
        personal_acct = key
        personal_balance = balance
    else:
        print('New account')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
print(f'Personal Account: {personal_balance}')
print(f'Rollover Account: {rollover_balance}')

In [ ]:
held_tickers = {}
for acct in accounts['response']['accounts']['accountsummary']:
    all_positions = [] 
    for position in acct['accountholdings']['holding']:
        position_sub = []
        position_sub.append(position['displaydata']['symbol'])
        position_sub.append(position['displaydata']['assetclass'])
        position_sub.append(position['displaydata']['qty'])
        position_sub.append(position['displaydata']['marketvalue'])
        position_sub.append(position['displaydata']['accounttype'])
        all_positions.append(position_sub)

    if acct['account'][-2:] == '53':
        held_tickers[acct['account']] = all_positions
    elif acct['account'][-2:] == '81':
        held_tickers[acct['account']] = all_positions
        
    else:
        held_tickers['other'] = all_positions
    
        

## Trading History

In [ ]:
all_trades = {}
for acct in my_accounts.keys():
    trades = []
    history = requests.get(uri_hist + acct + uri_hist2, auth = auth).json()
    for trade in history['response']['transactions']['transaction']:
        sub_trade = []
        if trade['transaction']['security']['sectyp'] == 'OPT' and trade['desc'][-3:] == 'Put':
            sub_trade.append(trade['desc'][-3:])
        elif trade['transaction']['security']['sectyp'] == 'OPT' and trade['desc'][-4:] == 'Call':
            sub_trade.append(trade['desc'][-4:])
        elif trade['transaction']['security']['sectyp'] == 'CS' and trade['activity'] in ['Trade', 'Assigned']:
            sub_trade.append('Stock')
        elif trade['transaction']['security']['sectyp'] == 'CS' and trade['activity'] == 'Dividend':
            sub_trade.append('Divident')
        else:
            sub_trade.append('Other')
        sub_trade.append(trade['symbol'])
        sub_trade.append(trade['activity'])
        sub_trade.append(trade['date'].split('T')[0])
        sub_trade.append(trade['transaction']['quantity'])
        sub_trade.append(trade['transaction']['price'])
        sub_trade.append(trade['transaction']['commission'])
        sub_trade.append(trade['transaction']['fee'])
        sub_trade.append(trade['transaction']['secfee'])
        sub_trade.append(trade['amount'])
        trades.append(sub_trade)
    all_trades[acct] = trades
    
all_traded_ticks = {}
for acct, trades in all_trades.items():
    sub_traded_ticks = []
    for trade in trades:
        if trade[0] == 'Stock' and trade[1] not in sub_traded_ticks:
            sub_traded_ticks.append(trade[1])
    all_traded_ticks[acct] = sub_traded_ticks

In [ ]:
for key, trades in all_trades.items():
    for trade in trades:
        if trade[1] != '':
            print(f'{key} {trade}')

In [ ]:


trades = []
test_split = [trade for trade in all_trades['38746981'] if trade[1] in ['MQ', 'ROKU', 'V']]
for item in test_split:
    if item[0] not in set([trade[1] for trade in test_split]):
        if item[0] == 'Stock' and int(trade[4]) > 0:
            print(item)
            
    
    
        

In [ ]:
discard_pile = ['Divident']
used_indexes = {}
# ticker, purchase_date, breakeven_price, sell_date, sell_price
stock_positions = {}
used_indexes = {}
for acct in all_traded_ticks.keys():
    traded_tickers = all_traded_ticks[acct]
    sub_stock_positions = []
    sub_used_indexes = []
    for trade in all_trades[acct]:
        if trade[1] in traded_tickers:
            print(trade)
            
            
    

In [ ]:
discard_pile = ['Divident', 'Other']
used_indexes = {}
# ticker, purchase_date, breakeven_price, sell_date, sell_price
stock_positions = {}
used_indexes = {}
for acct in all_traded_ticks.keys():
    traded_tickers = all_traded_ticks[acct]
    sub_stock_positions = []
    sub_used_indexes = []
    for trade in all_trades[acct]:
        if trade[1] in traded_tickers:
            sub_stack = []
            if trade[0] == 'Stock' and int(trade[4]) > 0:
                # This find the first
                curr_tick = trade[1]
                breakeven_price = float(trade[5])
                buy_qty = trade[4]
                buy_date = trade[3]
                sold_price = 0
                sold_date = 0
                sold_qty = 0
                buy_index = all_trades[acct].index(trade)
                curr_index = all_trades[acct].index(trade) + 1
                sub_used_indexes.append(buy_index)
                while curr_index < len(all_trades[acct]) - 1:
                    if all_trades[acct][curr_index][1] == curr_tick and all_trades[acct][curr_index][0] not in discard_pile:
                        breakeven_price += abs(float(all_trades[acct][curr_index][9])) / int(all_trades[acct][curr_index][4]) / 100
                        sub_used_indexes.append(curr_index)
                    curr_index += 1
                    if all_trades[acct][curr_index][0] == 'Stock' and all_trades[acct][curr_index][1] == curr_tick:
                        sold_price = all_trades[acct][curr_index][5]
                        sold_qty = all_trades[acct][curr_index][4]
                        sold_date = all_trades[acct][curr_index][3]
                        sold_index = curr_index
                        sub_used_indexes.append(sold_index)
                        curr_index = len(all_trades[acct])
                sub_stack.append(curr_tick)
                sub_stack.append(buy_date)
                sub_stack.append(sold_date)
                sub_stack.append(breakeven_price)
                sub_stack.append(sold_price)
                sub_stack.append(buy_qty)
                sub_stack.append(sold_qty)
                sub_stock_positions.append(sub_stack)
        stock_positions[acct] = sub_stock_positions
        sub_used_indexes.sort(reverse=True)
        used_indexes[acct] = sub_used_indexes

In [ ]:
for acct, trades in all_trades.items():
    if acct == '':
        for trade in trades:
            if trade[1] =='V':
                print(trade)

In [ ]:
# trades that didnt make it into the position rollup
all_residual = all_trades.copy()
for acct in my_accounts.keys():
    all_residual[acct] = [all_residual[acct].pop(index) for index in used_indexes[acct]]

In [ ]:
#separating open and closed positions
open_position, closed_position = {}, {}
for acct in my_accounts.keys():
    active_tickers =  [item[0] for item in held_tickers[acct] if item[1] == "Stock"]
    sub_open, sub_closed = [], []
    for position in stock_positions[acct]:
        if position[0] in active_tickers:
            sub_open.append(position)
        else:
            sub_closed.append(position)
    open_position[acct] = sub_open
    closed_position[acct] = sub_closed

trades_output = []
for key, trades in open_position.items():
    if key[-2:] == '81':
        account = 'Personal'
    elif key[-2:] == '53':
        account = 'Rollover'
    else:
        account = 'None'
    for trade in trades:
        sub_trades_output = []
        sub_trades_output.append(account)
        sub_trades_output.append(trade[0])
        sub_trades_output.append(trade[1])
        sub_trades_output.append(trade[3])
        sub_trades_output.append(trade[5])
        trades_output.append(sub_trades_output)
open_positions_df = pd.DataFrame(trades_output, columns=['Account', 'Ticker', 'Purchase Date',
                                                        'Breakeven Price', 'Holding Qantity'])

closed_trades = []
for key, trades in closed_position.items():
    if key[-2:] == '81':
        account = 'Personal'
    elif key[-2:] == '53':
        account = 'Rollover'
    else:
        account = 'None'
    for trade in trades:
        sub_trades_output = []
        sub_trades_output.append(account)
        sub_trades_output.append(trade[0])
        sub_trades_output.append(trade[1])
        sub_trades_output.append(trade[2])
        sub_trades_output.append(trade[3])
        sub_trades_output.append(trade[4])
        sub_trades_output.append(trade[5])
        sub_trades_output.append(trade[6])
        sub_trades_output.append(float(trade[4]) - float(trade[3]))
        sub_trades_output.append(float(trade[6]) + float(trade[5]))
        closed_trades.append(sub_trades_output)
closed_trades_df = pd.DataFrame(closed_trades, columns = ['Account', 'Ticker', 'Purchase Date', 'Sell Date',
                                                         'Breakeven Price', 'Sell Price', 'Hold Quantity',
                                                         'Sold Quantity', 'Profit/Loss', 'Remaining Quantity'])


In [ ]:
closed_trades_df[closed_trades_df['Ticker'] == 'V']

In [ ]:
open_positions_df

## Getting Puts for investment

In [4]:
# getting todays date
current_day = datetime.date.today()

symbol_dict = {}
for s in my_symbols:
    # pulling dates for curreny symbol
    dates = requests.get(uri_option_exp + symbol_call + s, auth = auth).json()
    expiration_dates = dates['response']['expirationdates']['date']

    exp_date_list = []
    for i in expiration_dates:    
        if (datetime.datetime.strptime(i, '%Y-%m-%d').date() - current_day).days <= 12:
            exp_date_list.append(i.split('-')[0] + i.split('-')[1] + i.split('-')[2])
            
    symbol_dict[s] = exp_date_list
    
    # pulling option chain for specific date and multiple tickers
i = 1
option_list = []
for sym, exp_dates in symbol_dict.items():
    stock_price = requests.get(uri_stock + stock_symbol + sym, auth = auth).json()
    stock_ask = stock_price['response']['quotes']['quote']['ask']
    stock_bid = stock_price['response']['quotes']['quote']['bid']
    stock_last = stock_price['response']['quotes']['quote']['last']
    for exp_date in exp_dates:
        r = requests.get(uri_option + symbol_call + sym + date_qry + exp_date, auth = auth).json()
        my_calls = r['response']['quotes']['quote']
        for my_call in my_calls:
            option_sublist = []
            option_sublist.append(my_call['undersymbol'])
            option_sublist.append(stock_last)
            option_sublist.append(stock_ask)
            option_sublist.append(stock_bid)
            option_sublist.append(current_day)
            option_sublist.append(exp_date[0:4] + '-' + exp_date[4:6] + '-' + exp_date[6:8])
            option_sublist.append(my_call['ask'])
            option_sublist.append(my_call['bid'])
            option_sublist.append(my_call['strikeprice'])
            option_sublist.append(my_call['imp_Volatility'])
            option_sublist.append(my_call['idelta'])
            option_sublist.append(my_call['openinterest'])
            option_sublist.append(my_call['put_call'])
            option_list.append(option_sublist)

options_df = pd.DataFrame(option_list, columns =['under_ticker', 'stock_current', 'stock_ask', 
                                                 'stock_bid', 'current_day', 'expiration_date', 'option_ask',
                                                 'option_bid', 'strike', 'volatility', 'delta',
                                                 'open_interest', 'option_type'])

#picking just the options for start of the wheel
puts = options_df[(options_df['option_type']=='put') & 
           (options_df['delta'].astype(float) <= -0.25) & (options_df['delta'].astype(float) >= -0.4)]

# adjusting the column types in the dataframe
puts.loc[:, 'stock_current'] = puts.loc[:, 'stock_current'].astype('float') 
puts.loc[:, 'stock_ask'] = puts.loc[:, 'stock_ask'].astype('float') 
puts.loc[:, 'stock_bid'] = puts.loc[:, 'stock_bid'].astype('float') 
puts.loc[:, 'option_ask'] = puts.loc[:, 'option_ask'].astype('float') 
puts.loc[:, 'option_bid'] = puts.loc[:, 'option_bid'].astype('float') 
puts.loc[:, 'strike'] = puts.loc[:, 'strike'].astype('float') 
puts.loc[:, 'volatility'] = puts.loc[:, 'volatility'].astype('float') 
puts.loc[:, 'delta'] = puts.loc[:, 'delta'].astype('float') 
puts.loc[:, 'open_interest'] = puts.loc[:, 'open_interest'].astype('int') 
puts.loc[:, 'current_day'] = puts.loc[:, 'current_day'].apply(pd.to_datetime)
puts.loc[:, 'expiration_date'] = pd.to_datetime(puts.loc[:, 'expiration_date'])
puts.loc[:, 'exp_date'] = pd.to_datetime(puts.loc[:, 'expiration_date']).apply(lambda x: x.date())

#building the analysis columns into the datafram
puts.loc[:, 'days_to_exp'] =  (puts.loc[:, 'expiration_date']\
                                   - puts.loc[:, 'current_day']) / np.timedelta64(1, 'D')
puts.loc[:, 'price_change'] = ((puts.loc[:, 'stock_current'] \
                                    / puts.loc[:, 'strike']) - 1).map('{:.2%}'.format)

puts.loc[:, 'annul_premium_ask'] = ((((puts.loc[:, 'option_ask'] * 100) \
                                           / (puts.loc[:, 'days_to_exp'] +1)) * 365) \
                                         / (puts.loc[:, 'strike'] * 100)).map('{:.2%}'.format)
puts.loc[:, 'annul_premium_bid'] = ((((puts.loc[:, 'option_bid'] * 100) \
                                           / (puts.loc[:, 'days_to_exp'] +1)) * 365) \
                                         / (puts.loc[:, 'strike'] * 100)).map('{:.2%}'.format)

# sorting the final output
target_puts = puts[['under_ticker', 'stock_current', 'strike', 'price_change', 'exp_date', 'days_to_exp', 
          'option_ask', 'option_bid', 'annul_premium_ask', 'annul_premium_bid']]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Outputs

In [ ]:
target_puts

In [ ]:
# picking highest return for each ticker
select_puts = []
for sym in my_symbols:
    max_return = 0
    for row in target_puts.iterrows():
        curr = float(row[1]['annul_premium_ask'].strip('%'))
        if row[1]['under_ticker'].lower() == sym and curr > max_return :
            max_return = curr
            ticker_max = row[0]
    select_puts.append(ticker_max)
target_puts.loc[select_puts, :].transpose()